In [1]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# LCEL接口
- 输入格式
- 输出格式
- 8种不同的接口方式
<hr>

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-3.5-turbo",
)
prompt = ChatPromptTemplate.from_template("给我讲一个关于{topic}的笑话")
chain = prompt | model

input schema

In [4]:
#prompt.
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [5]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [6]:
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'strin

Output Schema

In [7]:
# The output schema of the chain is the output schema of its last part, in this case a ChatModel, which outputs a ChatMessage
chain.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'ai',
     'enum': ['ai'],
     'type': 'string'},
    'name': {'title': 'Name', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'},
    'example': {'title': 'Example', 'default': False, 'type': 'boolean'}

Stream（流式）

In [8]:
for s in chain.stream({"topic": "熊"}):
    print(s.content, end="", flush=True)

为什么熊喜欢在树林里打篮球？

因为他们喜欢熊篷！😄

Invoke

In [9]:
chain.invoke({"topic": "熊"})

AIMessage(content='为什么熊不喜欢下雨天？\n因为它会变成“湿熊”！😄🐻', response_metadata={'finish_reason': 'stop', 'logprobs': None})

Batch

In [10]:
chain.batch([{"topic": "熊"}, {"topic": "猫"}])

[AIMessage(content='为什么熊不喜欢冬天？\n\n因为它们的冰淇淋总是融化！哈哈哈！', response_metadata={'finish_reason': 'stop', 'logprobs': None}),
 AIMessage(content='为什么猫喜欢打猎？\n因为他们喜欢尖叫！', response_metadata={'finish_reason': 'stop', 'logprobs': None})]

In [11]:
#max_concurrency控制并发数
chain.batch([{"topic": "熊"}, {"topic": "猫"}, {"topic": "狗"}], config={"max_concurrency": 5})

[AIMessage(content='为什么熊不喜欢在雨天出门？\n因为它怕变成“湿熊”！😄🐻', response_metadata={'finish_reason': 'stop', 'logprobs': None}),
 AIMessage(content='为什么猫喜欢在键盘上踩来踩去？\n\n因为它们喜欢按键！', response_metadata={'finish_reason': 'stop', 'logprobs': None}),
 AIMessage(content='当狗狗去看牙医时，它们会被问到：“你是不是每天都刷牙？”狗狗回答：“是的，但我还是用爪子擦地板。”哈哈哈。', response_metadata={'finish_reason': 'stop', 'logprobs': None})]

Async Stream 异步

In [12]:
async for s in chain.astream({"topic": "女人"}):
    print(s.content, end="", flush=True)

为什么女人喜欢购物？因为她们觉得买东西比减肥更能让自己开心！哈哈哈哈！

Async Invoke

In [13]:
await chain.ainvoke({"topic": "男人"})

AIMessage(content='为什么男人喜欢开车？因为开车的时候，他们可以不用停下来问路。', response_metadata={'finish_reason': 'stop', 'logprobs': None})

Async Batch

In [14]:
await chain.abatch([{"topic": "熊"},{"topic": "女人"}])

[AIMessage(content='为什么熊不喜欢雨天？\n\n因为它们怕变成“憨憨”！哈哈哈哈！', response_metadata={'finish_reason': 'stop', 'logprobs': None}),
 AIMessage(content='好的，这里有一个关于女人的笑话：\n\n为什么女人喜欢购物？\n因为购物可以让她们忘记所有的烦恼，直到收到账单时又开始烦恼起来。😄', response_metadata={'finish_reason': 'stop', 'logprobs': None})]

异步获取中间步骤

In [15]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

template = """基于下面的上下文来回答问题:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["柯基犬是一种中型家庭宠物犬"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [16]:
async for chunk in retrieval_chain.astream_log(
    "柯基是什么?", include_names=["Docs"]
):
    print("-" * 40)
    print(chunk)

----------------------------------------
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '756a766b-029b-4026-86a4-0b04cf7bf52f',
            'logs': {},
            'name': 'RunnableSequence',
            'streamed_output': [],
            'type': 'chain'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs',
  'value': {'end_time': None,
            'final_output': None,
            'id': '40e13b9f-893f-40e0-8cdf-5ccb45993a0f',
            'metadata': {},
            'name': 'Docs',
            'start_time': '2024-03-14T09:47:23.960+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['map:key:context', 'FAISS', 'OpenAIEmbeddings'],
            'type': 'retriever'}})
----------------------------------------
RunLogPatch({'op': 'add', 'path': '/logs/Docs/final_output', 'value': None},
 {'op': 'add',
  'path': '/logs/Docs/end_time',
  'value': '2024-03-

AuthenticationError: Error code: 401 - {'error': {'message': 'This API key is not authorized to access this model(`text-embedding-ada-002`) or the model you request does not exist. Please check or update your API key permissions, or check if the model name you requested is correct. (request id: 2024031409472452867036488463474)', 'type': 'upstream_error', 'param': '401', 'code': 'bad_response_status_code'}}

只看状态值

In [ ]:

async for chunk in retrieval_chain.astream_log(
    "柯基是什么?", include_names=["Docs"], diff=False
):
    print("-" * 70)
    print(chunk)

----------------------------------------------------------------------
RunLog({'final_output': None,
 'id': '87926d3f-6e27-4c38-8e3b-5635eaa26a65',
 'logs': {},
 'name': 'RunnableSequence',
 'streamed_output': [],
 'type': 'chain'})
----------------------------------------------------------------------
RunLog({'final_output': None,
 'id': '87926d3f-6e27-4c38-8e3b-5635eaa26a65',
 'logs': {'Docs': {'end_time': None,
                   'final_output': None,
                   'id': '869cd627-f35a-4b75-8900-cad2cf826903',
                   'metadata': {},
                   'name': 'Docs',
                   'start_time': '2024-02-16T03:25:39.457+00:00',
                   'streamed_output': [],
                   'streamed_output_str': [],
                   'tags': ['map:key:context', 'FAISS', 'OpenAIEmbeddings'],
                   'type': 'retriever'}},
 'name': 'RunnableSequence',
 'streamed_output': [],
 'type': 'chain'})
-------------------------------------------------------------

并行支持

In [ ]:
from langchain_core.runnables import RunnableParallel

chain1 = ChatPromptTemplate.from_template("给我讲一个关于{topic}的笑话") | model
chain2 = (
    ChatPromptTemplate.from_template("写两行关于{topic}的诗歌")
    | model
)
combined = RunnableParallel(joke=chain1, poem=chain2)

In [ ]:
%%time
chain1.invoke({"topic": "熊"})

CPU times: user 11.8 ms, sys: 9.75 ms, total: 21.6 ms
Wall time: 8.46 s


AIMessage(content='好的，这是一个关于熊的笑话：\n\n有一天，一只小熊和一只大熊在森林里散步。突然，小熊看到了一只兔子正在忙碌地搬运胡萝卜。小熊很好奇，就走过去问兔子：“嘿，兔子，你在干什么呢？”\n\n兔子回答说：“我在准备过冬，搬运胡萝卜到我的洞里。”\n\n小熊顿时眼前一亮，说：“哇，这是个好主意！我也要去搬运食物存起来。”\n\n于是小熊和大熊一起去寻找食物。他们找到了一堆蜂蜜，想要把它们都搬回洞里。但是，由于蜂蜜太黏稠，小熊和大熊都被粘住了，无法动弹。\n\n这时，兔子从远处走过来，看到了这一幕，忍不住笑了起来。\n\n小熊生气地问：“兔子，你为什么笑？”\n\n兔子笑着说：“因为你们这样，看起来就像两只大糖葫芦！”\n\n听到这个，小熊和大熊也忍不住笑了起来，他们意识到自己的可笑模样。\n\n最终，兔子帮助小熊和大熊脱离了黏蜜的困境，三个朋友一起度过了愉快的日子。\n\n这个笑话希望能给你带来一些欢乐！')

In [ ]:
%%time
chain2.invoke({"topic": "熊"})

CPU times: user 15.5 ms, sys: 3.32 ms, total: 18.9 ms
Wall time: 2.06 s


AIMessage(content='熊啊熊，森林中的王者，\n毛茸茸，力量无穷，让人敬畏。')

并行执行

In [ ]:
%%time
combined.invoke({"topic": "熊"})

CPU times: user 20.1 ms, sys: 3.56 ms, total: 23.7 ms
Wall time: 4.3 s


{'joke': AIMessage(content='好的，这里有一个关于熊的笑话：\n\n有一天，一只熊走进了一家餐馆。他坐在吧台前，对着酒保说：“我想要一杯……啊，忘了，我还是不要喝酒了。”酒保很好奇地问：“为什么？你是不是戒酒了？”熊摇了摇头，说：“不，其实是因为每次我喝酒后，我都会变得非常凶猛。”酒保好奇地问：“真的吗？那你是怎么知道的？”熊回答道：“因为每次我喝酒后，我都会去找猎人打架！”'),
 'poem': AIMessage(content='大熊在山间踏步行，\n森林中，威武似王者。')}

并行批处理，适用于大量生成

In [ ]:
%%time
chain1.batch([{"topic": "熊"}, {"topic": "猫"}])

CPU times: user 18.9 ms, sys: 3.17 ms, total: 22 ms
Wall time: 3.56 s


[AIMessage(content='当然，这是一个关于熊的笑话：\n\n有一天，一只熊走进了一家餐厅，坐在了一张桌子旁边。侍者走过来，询问熊说：“你想要点什么？” 熊说：“给我一份……鱼和…………苹果派。” 侍者看了一会儿，然后问道：“没问题，但是为什么你这么喜欢吃鱼和苹果派呢？” 熊回答说：“因为我是一只熊啊！”'),
 AIMessage(content='当猫遇到老鼠，老鼠问猫："你怕我吗？"猫回答说："怕，怕你长得比我还丑！"')]

In [ ]:
%%time
chain2.batch([{"topic": "熊"}, {"topic": "猫"}])

CPU times: user 19.9 ms, sys: 3.44 ms, total: 23.4 ms
Wall time: 3.11 s


[AIMessage(content='大熊展开双臂，迎风舞动如云蒸霞蔚。\n巍峨身姿挺拔，守护山林万物生机绚。'),
 AIMessage(content='闲卧红砖上，猫儿慵懒眯眼眸。\n蓬松尾巴飘飘扬，优雅姿态与我相伴。')]

并行执行

In [ ]:
%%time
combined.batch([{"topic": "熊"}, {"topic": "猫"}])

CPU times: user 35.7 ms, sys: 4.79 ms, total: 40.4 ms
Wall time: 5.32 s


[{'joke': AIMessage(content='当然！这是一个关于熊的笑话：\n\n有一天，一只小熊走进了一个酒吧。他走到吧台前，说：“请问，这里有熊吗？”吧台上的人们都愣住了，然后大笑起来。\n\n其中一个人回答道：“这里没有熊，只有人。”\n\n小熊感到非常失落，于是黯然离去。过了一段时间，小熊又回到了同一个酒吧。这次，他走到一个不同的吧台前，再次问道：“请问，这里有熊吗？”\n\n这次，人们没有大笑，而是纷纷低下头，不敢看小熊。小熊感到非常困惑，他问：“为什么你们这次不笑了？”\n\n吧台上的人们小声回答：“因为我们是熊。”\n\n希望这个笑话能让你开心！'),
  'poem': AIMessage(content='森林深处，熊呼啸，\n强壮的身躯，展露豪情。')},
 {'joke': AIMessage(content='当然！这是一个关于猫的笑话：\n\n有一天，一只猫走进了一家餐厅，他坐在了一张桌子旁边的椅子上。服务员非常惊讶地走过去，问道：“对不起，先生，我们这里不允许猫进来。”\n\n猫瞪大眼睛，回答说：“没关系，我只是想尝尝你们的鱼。”\n\n服务员笑了笑，说道：“非常抱歉，先生，我们的鱼菜单只供人类点餐。”\n\n猫眯起眼睛，微笑着说：“那没关系，我只要一条不会说话的鱼。”\n\n服务员无奈地笑了笑，点了一份鱼菜单放在猫面前。猫看了一眼菜单，然后把它推倒在地上。\n\n服务员惊讶地问：“为什么你这样做？”\n\n猫深情地看着服务员说：“因为我更喜欢鱼菜单的素材。”'),
  'poem': AIMessage(content='软毛猫儿眯着眼，悠闲自在享宁静。\n灵动身姿如舞蹈，温顺可爱似天使。')}]